# 📊 **Adidas Stock Data Analysis & Insights**

## 📝 **Metadata**

**Notebook Title:** Adidas Stock Data Analysis & Insights  
**Author:** Muhammad Hassan Saboor  
**Date Created:** January 2025  
**Last Updated:** January 2025  
**Tools Used:** Python, Pandas, Plotly, Statsmodels  
**Objective:** To perform exploratory data analysis (EDA) on Adidas stock data, uncover key insights, and visualize various stock-related metrics.

---

## 🔍 **Actions Performed in this Notebook**

1. **Data Loading & Preprocessing**  
   - Loaded Adidas stock data.
   - Converted the 'Date' column to datetime format for time-series analysis.

2. **General Analysis**  
   - Summary statistics for key columns.
   - Identification of outliers in stock prices and trading volume.
   
3. **Trend Analysis**  
   - Analyzed yearly and monthly trends in adjusted close prices.
   - Investigated volatility patterns and trading volume across time.

4. **Correlations**  
   - Studied correlations between stock prices (High, Low, Open, Close, Adj_Close).
   - Analyzed relationships between Volume and price changes.

5. **Performance Metrics**  
   - Calculated daily percentage changes and identified best/worst performing days.
   - Visualized moving averages (50-day, 100-day, 200-day).
   - Analyzed price spread between High and Low prices.

6. **Comparative Analysis**  
   - Investigated Open vs. Close price relationships.
   - Examined how often Close price is close to High/Low prices.

7. **Key Events**  
   - Identified peaks and troughs in adjusted close prices.
   - Investigated days with unusually high trading volume.

8. **Visualization Insights**  
   - Created candlestick charts for detailed stock movements.
   - Overlaid trading volume on price movements.
   - Generated heatmaps for monthly/weekly performance trends.
   - Plotted rolling standard deviation to capture volatility.
   - Visualized Volume vs. price changes using scatter plots.

9. **Seasonality & Day-of-Week Effects**  
   - Performed seasonal decomposition to analyze trends, seasonality, and residuals.
   - Investigated if certain days of the week have consistent price patterns.

10. **Price Prediction Insights**  
    - Analyzed patterns such as high trading volume leading to price spikes.

---

## 📊 **Columns in the Dataset**

| Column Name   | Description                                    | Data Type |
| ------------- | ---------------------------------------------- | --------- |
| **Date**      | Date of stock record                           | `object`  |
| **Adj_Close** | Adjusted close price after splits/dividends    | `float64` |
| **Close**     | Closing price of the stock                     | `float64` |
| **High**      | Highest price during the trading day           | `float64` |
| **Low**       | Lowest price during the trading day            | `float64` |
| **Open**      | Opening price of the stock                     | `float64` |
| **Volume**    | Total number of shares traded during the day   | `int64`   |

---

Thank you for exploring this notebook! Feel free to reach out for any questions or feedback. 😊


# **📚 Importing Libraries**

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings

# **⚙️ Basic Important Settings**

In [2]:
pio.renderers.default = 'iframe'
warnings.filterwarnings("ignore")

# **📥 Loading the Dataset**

In [3]:
df = pd.read_csv("/kaggle/input/adidas-stock-data-2006-2024/Adidas_Data.csv")

# **📊 Exploring the Dataset**

In [4]:
df.head()

,Date,Adj_Close,Close,High,Low,Open,Volume
0,2006-05-31,19.954313,24.9625,24.9750,24.962500,24.9625,28000
1,2006-06-01,19.834412,24.8125,24.8125,24.812500,24.8125,1600
2,2006-06-02,20.214109,25.2875,25.2875,25.250000,25.2875,1600
3,2006-06-05,20.184134,25.2500,25.2500,25.250000,25.2500,2000
4,2006-06-06,19.534645,24.4375,24.6500,24.075001,24.4375,10400


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4679 entries, 0 to 4678
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       4679 non-null   object 
 1   Adj_Close  4679 non-null   float64
 2   Close      4679 non-null   float64
 3   High       4679 non-null   float64
 4   Low        4679 non-null   float64
 5   Open       4679 non-null   float64
 6   Volume     4679 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 256.0+ KB


# **🔍 Exploratory Data Analysis (EDA)**

# **General Analysis**

In [6]:
df[['High', 'Low', 'Open', 'Close', 'Adj_Close', 'Volume']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
High,4679.0,74.375260,47.435302,14.000000,33.335001,55.740002,112.825001,1.994400e+02
Low,4679.0,73.303073,46.832085,12.950000,32.740000,55.000000,111.190002,1.962800e+02
Open,4679.0,73.849084,47.163218,13.650000,33.100000,55.389999,112.120003,1.966800e+02
Close,4679.0,73.874483,47.156565,13.000000,33.049999,55.500000,111.924999,1.986800e+02
Adj_Close,4679.0,69.311244,47.241404,10.631274,27.587647,49.055298,107.056389,1.933676e+02
Volume,4679.0,60613.336183,102978.972903,0.000000,19100.000000,31200.000000,58700.000000,2.511900e+06


In [7]:
df['Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [8]:
outliers = {}
for column in ['High', 'Low', 'Open', 'Close', 'Adj_Close', 'Volume']:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    outliers[column] = df[(df[column] < Q1 - 1.5 * IQR) | (df[column] > Q3 + 1.5 * IQR)]

# Highest and lowest stock prices and volumes by day/month
highest_prices = df.loc[df['Adj_Close'].idxmax()]
lowest_prices = df.loc[df['Adj_Close'].idxmin()]
highest_volume = df.loc[df['Volume'].idxmax()]
lowest_volume = df.loc[df['Volume'].idxmin()]

In [9]:
highest_prices

Date         2021-08-04 00:00:00
Adj_Close             193.367569
Close                 198.679993
High                  199.440002
Low                   196.279999
Open                  196.679993
Volume                    155700
Year                        2021
Month                          8
Day                            4
Name: 3821, dtype: object

In [10]:
lowest_prices

Date         2008-11-20 00:00:00
Adj_Close              10.631274
Close                       13.0
High                       14.05
Low                        12.95
Open                       13.65
Volume                     56500
Year                        2008
Month                         11
Day                           20
Name: 625, dtype: object

In [11]:
highest_volume

Date         2009-05-06 00:00:00
Adj_Close              13.809011
Close                       16.6
High                   16.950001
Low                        16.25
Open                   16.950001
Volume                   2511900
Year                        2009
Month                          5
Day                            6
Name: 738, dtype: object

In [12]:
lowest_volume

Date         2006-11-24 00:00:00
Adj_Close              19.504667
Close                       24.4
High                        24.4
Low                         24.4
Open                        24.4
Volume                         0
Year                        2006
Month                         11
Day                           24
Name: 124, dtype: object

## **Monthly average adjusted close price**

In [13]:
monthly_avg = df.groupby(['Year', 'Month'])['Adj_Close'].mean().reset_index()
monthly_avg['Year-Month'] = monthly_avg['Year'].astype(str) + '-' + monthly_avg['Month'].astype(str).str.zfill(2)

fig1 = px.line(
    monthly_avg,
    x='Year-Month',
    y='Adj_Close',
    title='Monthly Average Adjusted Close Price',
    labels={'Adj_Close': 'Average Adj Close Price', 'Year-Month': 'Year-Month'}
)
fig1.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig1.show()

## **Volume over time**

In [14]:
fig2 = px.bar(
    df,
    x='Date',
    y='Volume',
    title='Volume Over Time',
    labels={'Volume': 'Trading Volume', 'Date': 'Date'},
    template='plotly_dark'
)
fig2.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig2.show()

# **Trend Analysis**

In [15]:
# Calculating additional metrics for analysis
df['Daily_Change'] = df['High'] - df['Low']

## **Yearly trends in Adj_Close**

In [16]:
yearly_trends = df.groupby('Year')['Adj_Close'].mean().reset_index()

fig1 = px.line(
    yearly_trends,
    x='Year',
    y='Adj_Close',
    title='Yearly Average Adjusted Close Price',
    labels={'Adj_Close': 'Average Adj Close Price', 'Year': 'Year'},
)
fig1.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig1.show()

## **Monthly average Adj_Close price**

In [17]:
monthly_trends = df.groupby(['Year', 'Month'])['Adj_Close'].mean().reset_index()
monthly_trends['Year-Month'] = monthly_trends['Year'].astype(str) + '-' + monthly_trends['Month'].astype(str).str.zfill(2)

fig2 = px.line(
    monthly_trends,
    x='Year-Month',
    y='Adj_Close',
    title='Monthly Average Adjusted Close Price',
    labels={'Adj_Close': 'Average Adj Close Price', 'Year-Month': 'Year-Month'}
)
fig2.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig2.show()

## **Volatility analysis**

In [18]:
volatility = df.groupby('Year')['Daily_Change'].mean().reset_index()

fig3 = px.bar(
    volatility,
    x='Year',
    y='Daily_Change',
    title='Average Yearly Volatility (High - Low)',
    labels={'Daily_Change': 'Average Daily Price Change', 'Year': 'Year'}
)
fig3.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig3.show()

## **Trading volume analysis**

In [19]:
volume_analysis = df.groupby(['Year', 'Month'])['Volume'].sum().reset_index()
volume_analysis['Year-Month'] = volume_analysis['Year'].astype(str) + '-' + volume_analysis['Month'].astype(str).str.zfill(2)

fig4 = px.bar(
    volume_analysis,
    x='Year-Month',
    y='Volume',
    title='Monthly Trading Volume',
    labels={'Volume': 'Total Trading Volume', 'Year-Month': 'Year-Month'},
)
fig4.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig4.show()

# **Correlations**

In [20]:
price_columns = ['High', 'Low', 'Open', 'Close', 'Adj_Close']
price_correlation = df[price_columns].corr()

## **Correlation heatmap for price relationships**

In [21]:
fig1 = px.imshow(
    price_correlation,
    text_auto=True,
    title='Correlation Between Stock Prices',
    labels=dict(color='Correlation'),
    color_continuous_scale='viridis'
)
fig1.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig1.show()

## **Open vs. Close**

In [22]:
fig2 = px.scatter(
    df,
    x='Open',
    y='Close',
    trendline='ols',
    title='Relationship Between Open and Close Prices',
    labels={'Open': 'Opening Price', 'Close': 'Closing Price'}
)
fig2.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig2.show()

## **Volume vs. daily price changes (High - Low)**

In [23]:
df['Daily_Change'] = df['High'] - df['Low']
fig3 = px.scatter(
    df,
    x='Volume',
    y='Daily_Change',
    trendline='ols',
    title='Volume vs. Daily Price Change',
    labels={'Volume': 'Trading Volume', 'Daily_Change': 'Daily Price Change (High - Low)'}
)
fig3.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig3.show()

# **Performance Metrics**

## **Daily percentage changes in Adj_Close**

In [24]:
# Calculate daily percentage changes in Adj_Close
df['Daily_Percentage_Change'] = df['Adj_Close'].pct_change() * 100

# Identify best and worst-performing days
best_day = df.loc[df['Daily_Percentage_Change'].idxmax()]
worst_day = df.loc[df['Daily_Percentage_Change'].idxmin()]

# Average daily return and cumulative return
average_daily_return = df['Daily_Percentage_Change'].mean()
df['Cumulative_Return'] = (1 + df['Daily_Percentage_Change'] / 100).cumprod()

# Moving averages
df['MA_50'] = df['Adj_Close'].rolling(window=50).mean()
df['MA_100'] = df['Adj_Close'].rolling(window=100).mean()
df['MA_200'] = df['Adj_Close'].rolling(window=200).mean()

# Price spread
df['Price_Spread'] = df['High'] - df['Low']

In [25]:
fig1 = px.line(
    df,
    x='Date',
    y='Daily_Percentage_Change',
    title='Daily Percentage Change in Adj Close Price',
    labels={'Daily_Percentage_Change': 'Daily % Change', 'Date': 'Date'}
)
fig1.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig1.show()

## **Cumulative return**

In [26]:
fig2 = px.line(
    df,
    x='Date',
    y='Cumulative_Return',
    title='Cumulative Return Over Time',
    labels={'Cumulative_Return': 'Cumulative Return', 'Date': 'Date'}
)
fig2.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig3.show()

## **Moving averages**

In [27]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=df['Date'], y=df['Adj_Close'], mode='lines', name='Adj Close'))
fig3.add_trace(go.Scatter(x=df['Date'], y=df['MA_50'], mode='lines', name='50-Day MA'))
fig3.add_trace(go.Scatter(x=df['Date'], y=df['MA_100'], mode='lines', name='100-Day MA'))
fig3.add_trace(go.Scatter(x=df['Date'], y=df['MA_200'], mode='lines', name='200-Day MA'))
fig3.update_layout(
    title='Moving Averages and Adj Close Price',
    xaxis_title='Date',
    yaxis_title='Price',
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)
fig3.show()

## **Price spread**

In [28]:
fig4 = px.line(
    df,
    x='Date',
    y='Price_Spread',
    title='Daily Price Spread (High - Low)',
    labels={'Price_Spread': 'Price Spread', 'Date': 'Date'}
)
fig4.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig4.show()

# **Comparative Analysis**

In [29]:
# Calculate whether Close is higher or lower than Open
df['Close_Higher_Than_Open'] = df['Close'] > df['Open']

# Calculate average price difference between Open and Close
df['Price_Difference'] = df['Close'] - df['Open']
average_price_difference = df['Price_Difference'].mean()

# Calculate how often Close is close to High or Low
df['Close_Near_High'] = abs(df['Close'] - df['High']) <= 0.01 * df['High']
df['Close_Near_Low'] = abs(df['Close'] - df['Low']) <= 0.01 * df['Low']

# Proportion of days Close > Open
close_higher_count = df['Close_Higher_Than_Open'].sum()
total_days = len(df)
close_higher_percentage = (close_higher_count / total_days) * 100

## **Proportion of days Close > Open**

In [30]:
fig1 = px.pie(
    names=['Close > Open', 'Close ≤ Open'],
    values=[close_higher_count, total_days - close_higher_count],
    title='Proportion of Days Where Close is Higher Than Open',
)
fig1.update_traces(textinfo='percent', marker=dict(colors=['green', 'red']))
fig1.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig1.show()

## **Average price difference between Open and Close**

In [31]:
fig2 = px.histogram(
    df,
    x='Price_Difference',
    title='Distribution of Price Difference (Close - Open)',
    labels={'Price_Difference': 'Price Difference (Close - Open)'},
    color_discrete_sequence=['cyan']
)
fig2.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig2.show()

## **Days Close is near High or Low**

In [32]:
close_near_high_count = df['Close_Near_High'].sum()
close_near_low_count = df['Close_Near_Low'].sum()
fig3 = px.bar(
    x=['Close Near High', 'Close Near Low'],
    y=[close_near_high_count, close_near_low_count],
    title='Frequency of Close Being Near High or Low',
    labels={'x': 'Condition', 'y': 'Number of Days'},
    text=[close_near_high_count, close_near_low_count],
    color=['Close Near High', 'Close Near Low'],
    color_discrete_sequence=['blue', 'orange']
)
fig3.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig3.show()

# **Key Events**

In [33]:
# Identify peaks and troughs in Adj_Close
max_adj_close_date = df.loc[df['Adj_Close'].idxmax(), 'Date']
max_adj_close_value = df['Adj_Close'].max()
min_adj_close_date = df.loc[df['Adj_Close'].idxmin(), 'Date']
min_adj_close_value = df['Adj_Close'].min()

# Identify volume spikes (above 95th percentile)
volume_spike_threshold = df['Volume'].quantile(0.95)
volume_spike_data = df[df['Volume'] > volume_spike_threshold]

## **Peaks and Troughs in Adj_Close**

In [34]:
fig1 = px.line(
    df,
    x='Date',
    y='Adj_Close',
    title='Adj_Close Peaks and Troughs',
    labels={'Adj_Close': 'Adjusted Close Price'},
)
fig1.add_trace(
    go.Scatter(
        x=[max_adj_close_date, min_adj_close_date],
        y=[max_adj_close_value, min_adj_close_value],
        mode='markers+text',
        marker=dict(size=10, color='red'),
        text=["Peak", "Trough"],
        textposition="top center"
    )
)
fig1.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig1.show()

## **Volume Spikes**

In [35]:
fig2 = px.scatter(
    volume_spike_data,
    x='Date',
    y='Volume',
    title='Volume Spikes Above 95th Percentile',
    labels={'Volume': 'Trading Volume'},
    color='Adj_Close',
    color_continuous_scale='viridis'
)
fig2.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig2.show()

# **Visualization Insights**

In [36]:
fig1 = go.Figure(data=[go.Candlestick(
    x=df['Date'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    increasing_line_color='green', decreasing_line_color='red'
)])
fig1.update_layout(
    title='Candlestick Chart of Adidas Stock',
    xaxis_title='Date',
    yaxis_title='Price',
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)
fig1.show()

## **Volume Overlay**

In [37]:
fig2 = go.Figure()

# Add Candlestick Chart
fig2.add_trace(go.Candlestick(
    x=df['Date'],
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    increasing_line_color='green', decreasing_line_color='red'
))

# Add Volume Overlay
fig2.add_trace(go.Bar(
    x=df['Date'],
    y=df['Volume'],
    name='Volume',
    marker=dict(color='rgba(255, 255, 255, 0.3)')
))

fig2.update_layout(
    title='Adidas Stock with Volume Overlay',
    xaxis_title='Date',
    yaxis_title='Price',
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)
fig2.show()

## **Heatmap for Monthly Performance Trends**

In [38]:
df['YearMonth'] = df['Date'].dt.to_period('M')
monthly_avg = df.groupby('YearMonth')['Adj_Close'].mean().reset_index()
monthly_avg['YearMonth'] = monthly_avg['YearMonth'].astype(str)

fig3 = px.imshow(
    monthly_avg.pivot_table(index='YearMonth', columns='YearMonth', values='Adj_Close'),
    title="Monthly Performance Heatmap",
    labels={'x': 'Month', 'y': 'Year'},
    color_continuous_scale='Viridis'
)
fig3.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)
fig3.show()

## **Rolling Standard Deviation (Volatility)**

In [39]:
df['Rolling_Std'] = df['Adj_Close'].rolling(window=30).std()

fig4 = px.line(
    df,
    x='Date',
    y='Rolling_Std',
    title='30-Day Rolling Standard Deviation (Volatility)',
    labels={'Rolling_Std': 'Rolling Std Dev'},
    color_discrete_sequence=['cyan']
)
fig4.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)
fig4.show()

## **Volume vs. Daily Price Changes**

In [40]:
df['Daily_Price_Change'] = df['Adj_Close'].diff()

fig5 = px.scatter(
    df,
    x='Volume',
    y='Daily_Price_Change',
    title='Volume vs. Daily Price Changes',
    labels={'Volume': 'Volume', 'Daily_Price_Change': 'Daily Price Change'},
    color='Adj_Close',
    color_continuous_scale='Viridis'
)
fig5.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)
fig5.show()

# **Advanced Insights**

In [41]:
# Seasonal Decomposition of Adj_Close
df.set_index('Date', inplace=True)
seasonal_decomposition = seasonal_decompose(df['Adj_Close'], model='multiplicative', period=365)

## **Trend, Seasonal, Residual Components**

In [42]:
fig1 = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

fig1.add_trace(go.Scatter(x=seasonal_decomposition.trend.index, y=seasonal_decomposition.trend, mode='lines', name='Trend'), row=1, col=1)
fig1.add_trace(go.Scatter(x=seasonal_decomposition.seasonal.index, y=seasonal_decomposition.seasonal, mode='lines', name='Seasonality'), row=2, col=1)
fig1.add_trace(go.Scatter(x=seasonal_decomposition.resid.index, y=seasonal_decomposition.resid, mode='lines', name='Residuals'), row=3, col=1)

fig1.update_layout(
    title='Seasonal Decomposition of Adidas Stock Adjusted Close',
    plot_bgcolor='black', paper_bgcolor='black', font_color='white'
)
fig1.show()

## **Day-of-the-Week Effects**

In [43]:
df['DayOfWeek'] = df.index.dayofweek
day_of_week_avg = df.groupby('DayOfWeek')['Adj_Close'].mean()

fig2 = px.bar(
    day_of_week_avg,
    x=day_of_week_avg.index,
    y=day_of_week_avg.values,
    title='Average Adjusted Close Price by Day of the Week',
    labels={'x': 'Day of Week', 'y': 'Average Adj_Close'},
    color=day_of_week_avg.values,
    color_continuous_scale='Viridis'
)
fig2.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig2.show()

## **Price Prediction Insights: Volume Leading to Price Spikes**

In [44]:
df['PriceChange'] = df['Adj_Close'].diff()
df['VolumeChange'] = df['Volume'].diff()

fig3 = px.scatter(
    df,
    x='VolumeChange',
    y='PriceChange',
    title='Volume Change vs. Price Change',
    labels={'VolumeChange': 'Change in Volume', 'PriceChange': 'Change in Price'},
    color='Adj_Close',
    color_continuous_scale='Viridis'
)
fig3.update_layout(plot_bgcolor='black', paper_bgcolor='black', font_color='white')
fig3.show()

### Thank You! 

Thank you for taking the time to explore and analyze the Adidas stock data with me. I hope the insights presented in this notebook were valuable and informative. 📊

Your feedback, questions, and suggestions are always welcome, so feel free to reach out! 💬

Happy analyzing, and best of luck with your data journey! 🚀😊
